In [1]:
# 표준 라이브러리
import os
import re
import json
import time
import urllib.parse
import urllib.request
from datetime import datetime

# 외부 라이브러리
import requests
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
from bs4 import BeautifulSoup

# NLTK 관련 다운로드
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# NLTK 모듈
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize, word_tokenize


# 네이버 API 클라이언트 ID와 시크릿
client_id = "4aM0BLbwSKf5jwggWUmb"
client_secret = "VG12t6mFAJ"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4호실-8\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\4호실-8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# URL 요청 함수: API에 요청을 보내고 응답을 반환
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
        else:
            return None
    except Exception as e:
        print(e)
        return None

# 네이버 뉴스 검색 API를 사용하여 뉴스를 검색하는 함수
def searchNaverNews(query, from_date, to_date, display=10, start=1):
    base_url = "https://openapi.naver.com/v1/search/news.json"
    query = urllib.parse.quote(query)
    url = f"{base_url}?query={query}&display={display}&start={start}&sort=date&startDate={from_date}&endDate={to_date}"
    
    response = getRequestUrl(url)
    if response is None:
        return None
    
    return json.loads(response)

# 네이버 뉴스 본문을 크롤링하는 함수
def fetchNaverNewsContent(url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # 네이버 뉴스 본문 추출 (주어진 HTML 구조에 맞게 수정)
            article_body = soup.find('article', {'id': 'dic_area', 'class': 'go_trans _article_content'})
            
            if article_body:
                # 불필요한 태그 제거 (예: 스크립트, 스타일 등)
                for unwanted in article_body(['script', 'style']):
                    unwanted.extract()
                return article_body.get_text(strip=True)
            else:
                return ""
        else:
            return ""
    except Exception as e:
        return str(e)



def getFilteredNews(query, date, keywords=None, display=10):
    # keywords가 주어지지 않으면 빈 리스트로 설정
    if keywords is None:
        keywords = []
    
    news_items = searchNaverNews(query, date, date, display)
    
    if news_items:
        parsed_news = []
        for item in news_items['items']:
            try:
                title = BeautifulSoup(item['title'], 'html.parser').get_text()
                link = item['link']
                description = BeautifulSoup(item['description'], 'html.parser').get_text()
                
                # 원문 기사 크롤링
                content = fetchNaverNewsContent(link)
                summary_content = textrank_summary(content)
                
                # title, link, description, content 중 하나라도 비어 있으면 해당 기사를 넘기기
                if not title or not link or not description or not content:
                    continue
                
                # 키워드 리스트가 비어 있지 않으면 필터링
                if keywords and not any(keyword in content for keyword in keywords):
                    continue
                
                parsed_news.append({
                    'title': title,
                    'link': link,
                    'description': description,
                    'content': content,
                    'summary_content': summary_content
                })
            
            except Exception as e:
                # 오류가 발생하면 해당 기사를 넘기고 다음 기사를 처리
                print(f"기사를 처리하는 중 오류 발생: {e}")
                continue
        
        return parsed_news
    else:
        print("뉴스 검색에 실패했습니다.")
        return []

# 뉴스 데이터를 엑셀 파일로 저장하는 함수
def saveNewsToExcel(news_data, file_name="filtered_naver_news.xlsx"):
    df = pd.DataFrame(news_data)
    df.to_excel(file_name, index=False, engine='openpyxl')
    print(f"뉴스 데이터를 '{file_name}' 파일로 저장했습니다.")

def sentence_similarity(sent1, sent2, stopwords=None):
	# 불용어가 주어지지 않으면 빈 리스트로 초기화
	if stopwords is None:
		stopwords = []
		
	# 문장을 모두 소문자로 변환
	sent1 = [word.lower() for word in sent1 if word not in stopwords]
	sent2 = [word.lower() for word in sent2 if word not in stopwords]
	
	# 두 문장에서 모든 단어를 모두 중복을 제거한 집합 생성
	all_words = list(set(sent1 + sent2))
	
	# 각 문장의 단어 등장 횟수를 기록할 벡터 초기화
	vector1 = [0] * len(all_words)
	vector2 = [0] * len(all_words)
	
	# 첫 번째 문장의 단어 등장 횟수 기록
	for word in sent1:
		if word in stopwords:
			continue # 불용어 무시
		vector1[all_words.index(word)] += 1
	
	# 두 번째 문장의 단어 등장 횟수 기록
	for word in sent2:
		if word in stopwords:
			continue
		vector2[all_words.index(word)] += 1
		
		# 코사인 유사도 계산을 위해 벡터를 이용하여 측정
		return 1 - cosine_distance(vector1, vector2)
     
def build_similarity_matrix(sentences, stop_words):

	# 문장 간 유사도 행렬 초기화
	similarity_matrix = np.zeros((len(sentences), len(sentences)))
	
	# 모든 문장 쌍에 대해 유사도 계산
	for idx1 in range(len(sentences)):
		for idx2 in range(len(sentences)):
			# 같은 문장인 경우 계산하지 않음
			if idx1 == idx2:
				continue
				
			# 문장 간 유사도 게산하여 행렬에 할당
			similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
			
	return similarity_matrix

def textrank_summary(text, num_sentences=3):
    # 입력 텍스트를 문장 단위로 구분
    sentences = sent_tokenize(text)
    
    # 불용어 제거
    stop_words = ['을', '를', '이', '가', '은', '는', '에', '의', '과', '와', '한', '들', '의']
    sentences = [word for word in sentences if word not in stop_words]
    
    # 문장 간 유사도 행렬 생성
    similarity_matrix = build_similarity_matrix(sentences, stop_words)
    
    # 페이지랭크 알고리즘을 사용하여 각 문장의 점수 계산
    scores = nx.pagerank(nx.from_numpy_array(similarity_matrix))
    
    # 문장을 페이지랭크 점수에 따라 내림차순으로 정렬
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    # 상위 num_sentences 만큼의 문장을 선택하여 요약 생성
    summary = ' '.join([sentence for score, sentence in ranked_sentences[:num_sentences]])
    
    return summary

def saveNewsFromQueriesToExcel(queries, date, file_name="filtered_news.xlsx", display=10):
    all_filtered_news = []
    
    for query in queries:
        print(f"뉴스를 가져오는 중: {query}")
        filtered_news = getFilteredNews(query, date, keywords=None, display=display)
        for news in filtered_news:
            all_filtered_news.append(news)
    
    # 결과를 데이터프레임으로 변환
    df = pd.DataFrame(all_filtered_news)
    
    # 엑셀 파일로 저장
    df.to_excel(file_name, index=False, engine='openpyxl')
    print(f"뉴스 데이터를 '{file_name}' 파일로 저장했습니다.")

# 증시 가져오는 코드~~~~~~~~~~~~~

In [4]:
# 쿼리 리스트와 현재 날짜 설정
queries = ['증시', '코스피', '나스닥', '다우존스', 'snp']
date = datetime.now().strftime("%Y%m%d")  # 현재 날짜를 "YYYYMMDD" 형식으로 변환

# 함수 호출
saveNewsFromQueriesToExcel(queries, date, file_name="../../chatbot/data/filtered_news1.xlsx", display=100)

뉴스를 가져오는 중: 증시


C:\Users\4호실-8\AppData\Local\Temp\ipykernel_13264\1598435418.py:66: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  title = BeautifulSoup(item['title'], 'html.parser').get_text()


뉴스를 가져오는 중: 코스피


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


뉴스를 가져오는 중: 나스닥
뉴스를 가져오는 중: 다우존스
뉴스를 가져오는 중: snp


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


뉴스 데이터를 '../../chatbot/data/filtered_news1.xlsx' 파일로 저장했습니다.


# 진석님 이 아래거 돌리시면 오늘의 뉴스 가져와요~~

In [ ]:
query_list = ["삼성전자", '코스피']
keywords = ["HBM", "파운드리", "QLED", "OLED", "엑시노스", "갤럭시", 
            "이재용", "삼성페이", "메모리", "가우스", "비스포크", '반도체', 
            '평택', '텍사스', '플립', '폴더', '낸드', '패키징', 'bspdn', 'BSPDN', 'GAA']

date = datetime.now().date()
all_filtered_news = []
for query in query_list:
    filtered_news = getFilteredNews(query, date, keywords, display=100)
    for news in filtered_news:
        all_filtered_news.append(news)

# 네이버 뉴스 검색 쿼리와 키워드 정의
query = "애플"
keywords = [
    "M1", "M2", "M3", "아이폰", "A17", "Apple Intelligence", 
    "비젼 프로", "비전 프로", "ios", "팀 쿡", 
    "앱스토어", "애플티비", "애플뮤직"
]


apple_filtered_news = getFilteredNews(query, date, keywords, display=100)
for news in filtered_news:
    all_filtered_news.append(news)

# 네이버 뉴스 검색 쿼리와 키워드 정의
query = "비트코인"
keywords = [
    "엘살바도르", "비트", "블록체인", "빗썸",
    "채굴", "그래픽 카드", "트럼프", "암호화폐", "탈중앙화",
    "알트", "이더리움", "래플", "도지", "루나"
]

bitcoin_filtered_news = getFilteredNews(query, date, keywords, display=100)
for news in filtered_news:
    all_filtered_news.append(news)

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(all_filtered_news)

# 엑셀 파일로 저장
output_file_name = "filtered_news_with_outcome.xlsx"
df.to_excel(output_file_name, index=False, engine='openpyxl')

df.head()  # 첫 몇 줄을 표시하여 결과 확인

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
C:\Users\4호실-8\AppData\Local\Temp\ipykernel_9444\1148065935.py:64: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  title = BeautifulSoup(item['title'], 'html.parser').get_text()
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


,title,link,description,content,summary_content
0,최신폰 전환지원금 '0원'…고가요금 부추기는 공시지원금만 2배,https://n.news.naver.com/mnews/article/374/000...,[기자] 통신 3사가 삼성전자 신형 폴더블폰 공시지원금을 20만 원대에서 50만 원...,[앵커]갤럭시 신형 폴더블폰 공시지원금이 2주 만에 두 배 올랐습니다.하지만 통신사...,사업자 간 경쟁이 보조금보다는 요금제나 차별적 혜택과 같은 서비스 중심으로 축이 전...
1,하락장서 선방한 '채권혼합형' ETF의 힘,https://n.news.naver.com/mnews/article/009/000...,"애플이 6.72% 하락할 때 'PLUS 애플채권혼합'은 3.73%만 떨어졌고, 다시...",엔비디아 한달새 20% 하락때8%만 떨어지며 손실 방어국공채도 종목 구성에 포함해폭...,이들 상품은 2022년 국내 증시에 처음 출시된 대표적인 단일종목 ETF다.단일종목...
2,[코스닥 공시] 씨엔지하이테크,https://n.news.naver.com/mnews/article/009/000...,◇씨엔지하이테크=삼성전자와 223억원 규모 반도체 제조장비 공급 계약 체결.,◇씨엔지하이테크=삼성전자와 223억원 규모 반도체 제조장비 공급 계약 체결.,◇씨엔지하이테크=삼성전자와 223억원 규모 반도체 제조장비 공급 계약 체결.
3,"3조 순매도한 외국인…""증시 떠나기보단 관망""",https://n.news.naver.com/mnews/article/011/000...,"외국인들은 6거래일 동안 삼성전자(-1조 8330억 원), SK하이닉스(-4720억...","코스피 반등에도 외국인은 '팔자'이달 R공포발 급락 후 6거래일 연속삼전, SK하이...",연합뉴스[서울경제]미국발 경기 침체 공포로 코스피가 급락한 이달 2일 이후 외국인투...
4,삼성 파운드리 수주 물량 늘어난다…후공정까지 넓히는 국내 'DSP' [소부장반...,https://n.news.naver.com/mnews/article/138/000...,"AI 반도체 시장 확장 대응에 나선 삼성전자가 파운드리(위탁 생산)부터 메모리, 패...",9일 서울 코엑스에서 열린 '삼성 파운드리 포럼(SFF 2024)'의 기조연설자로 ...,"이에 파운드리 설계뿐 아니라 패키징과 같은 후공정 작업, 테스트까지도 팹리스 고객사..."


In [ ]:
date_samsung_outcome_mapping = {
    '2000-07-28': '악재', '2008-10-16': '악재', '2000-07-24': '악재', '2002-05-10': '악재', '2000-08-07': '악재', 
    '2003-03-31': '악재', '2000-08-23': '악재', '2012-06-25': '악재', '2000-08-28': '악재', '2008-10-23': '악재', 
    '2016-09-12': '악재', '2000-11-28': '악재', '2004-05-10': '악재', '2001-07-09': '악재', '2000-11-13': '악재', 
    '2004-06-11': '악재', '2000-05-26': '악재', '2002-12-30': '악재', '2003-02-25': '악재', '2020-03-23': '악재', 
    '2003-09-22': '악재', '2008-07-25': '악재', '2013-06-07': '악재', '2012-05-16': '악재', '2009-01-15': '악재', 
    '2001-09-20': '악재', '2008-12-23': '악재', '2001-02-05': '악재', '2007-11-08': '악재', '2002-05-06': '악재', 
    '2020-03-19': '악재', '2004-05-17': '악재', '2011-08-18': '악재', '2000-10-02': '악재', '2008-10-24': '악재', 
    '2004-06-03': '악재', '2002-07-26': '악재', '2008-04-24': '악재', '2002-01-15': '악재', '2001-10-30': '악재', 
    '2007-07-16': '악재', '2000-10-09': '악재', '2000-12-20': '악재', '2007-08-01': '악재', '2003-09-15': '악재', 
    '2007-08-16': '악재', '2001-11-28': '악재', '2001-05-31': '악재', '2002-04-08': '악재', '2001-12-21': '악재', 
    '2017-11-27': '악재', '2011-11-10': '악재', '2003-01-29': '악재', '2003-01-09': '악재', '2005-11-22': '악재', 
    '2007-10-16': '악재', '2008-01-22': '악재', '2002-12-04': '악재', '2006-03-03': '악재', '2011-09-05': '악재', 
    '2001-07-06': '악재', '2000-07-27': '악재', '2004-05-06': '악재', '2018-10-11': '악재', '2007-07-27': '악재', 
    '2003-10-23': '악재', '2001-04-03': '악재', '2002-04-29': '악재', '2011-08-09': '악재', '2008-02-11': '악재', 
    '2008-12-02': '악재', '2012-05-18': '악재', '2009-04-08': '악재', '2014-01-02': '악재', '2020-06-15': '악재', 
    '2011-03-15': '악재', '2001-03-12': '악재', '2006-03-07': '악재', '2002-04-25': '악재', '2004-10-12': '악재', 
    '2001-03-27': '악재', '2016-01-04': '악재', '2000-05-23': '악재', '2000-09-21': '악재', '2002-07-19': '악재', 
    '2004-04-29': '악재', '2015-03-26': '악재', '2007-11-27': '악재', '2018-02-02': '악재', '2001-06-11': '악재', 
    '2002-10-30': '악재', '2001-06-05': '악재', '2001-02-23': '악재', '2001-03-09': '악재', '2000-12-14': '악재', 
    '2024-08-02': '악재', '2008-05-19': '악재', '2003-04-09': '악재', '2009-11-27': '악재', '2000-07-20': '악재', 
    '2020-08-20': '악재', '2002-07-10': '악재', '2000-04-24': '악재', '2011-03-07': '악재', '2009-01-23': '악재', 
    '2008-06-19': '악재', '2001-11-29': '악재', '2017-07-28': '악재', '2011-08-19': '악재', '2001-04-02': '악재', 
    '2020-04-01': '악재', '2020-03-11': '악재', '2002-03-15': '악재', '2019-05-09': '악재', '2012-12-21': '악재', 
    '2020-02-24': '악재', '2011-09-23': '악재', '2002-09-23': '악재', '2006-06-13': '악재', '2006-06-08': '악재',
    '2001-12-05': '호재', '2000-10-20': '호재', '2000-11-01': '호재', '2008-10-30': '호재', '2001-01-04': '호재', 
    '2000-11-27': '호재', '2000-04-28': '호재', '2002-02-14': '호재', '2000-09-20': '호재', '2009-01-28': '호재', 
    '2020-03-24': '호재', '2002-01-02': '호재', '2001-04-19': '호재', '2015-10-07': '호재', '2008-12-08': '호재', 
    '2004-01-09': '호재', '2000-06-14': '호재', '2001-03-23': '호재', '2000-12-13': '호재', '2008-09-19': '호재', 
    '2002-01-14': '호재', '2001-04-18': '호재', '2001-04-12': '호재', '2002-11-04': '호재', '2000-09-25': '호재', 
    '2000-08-09': '호재', '2000-06-09': '호재', '2001-11-14': '호재', '2008-10-27': '호재', '2001-03-14': '호재', 
    '2021-01-08': '호재', '2001-01-31': '호재', '2003-01-03': '호재', '2011-12-01': '호재', '2008-10-14': '호재', 
    '2001-10-11': '호재', '2002-05-14': '호재', '2000-08-16': '호재', '2001-10-24': '호재', '2000-06-20': '호재', 
    '2002-04-02': '호재', '2003-02-17': '호재', '2007-07-13': '호재', 
    '2011-09-07': '호재', '2004-10-04': '호재', '2000-12-11': '호재', '2000-05-30': '호재', '2000-10-19': '호재', 
    '2000-05-31': '호재', '2005-01-14': '호재', '2001-09-13': '호재', '2013-06-27': '호재', '2002-07-12': '호재', 
    '2002-10-18': '호재', '2002-10-28': '호재', '2000-05-08': '호재', '2023-09-01': '호재', '2007-07-06': '호재', 
    '2011-08-16': '호재', '2020-06-03': '호재', '2001-01-02': '호재', '2000-11-24': '호재', '2008-10-28': '호재', 
    '2009-02-04': '호재', '2020-03-20': '호재', '2001-07-19': '호재', '2008-01-17': '호재', '2024-03-20': '호재', 
    '2002-01-04': '호재', '2001-01-16': '호재', '2004-05-19': '호재', '2000-05-16': '호재', '2003-03-03': '호재', 
    '2001-04-30': '호재', '2001-01-12': '호재', '2008-05-29': '호재', '2009-07-06': '호재', '2003-04-07': '호재', 
    '2003-04-29': '호재', '2000-08-01': '호재', '2001-06-07': '호재', '2020-07-28': '호재', '2014-10-31': '호재', 
    '2004-04-06': '호재', '2002-10-14': '호재', '2002-11-15': '호재', '2020-12-24': '호재', '2001-03-07': '호재', 
    '2014-11-27': '호재', '2003-03-20': '호재', '2001-05-17': '호재', '2009-01-07': '호재', '2012-07-27': '호재', 
    '2021-11-22': '호재', '2007-12-06': '호재', '2012-06-07': '호재', '2014-11-10': '호재', '2011-11-11': '호재', 
    '2013-07-11': '호재', '2002-03-19': '호재', '2006-01-04': '호재', '2002-06-28': '호재', '2012-02-15': '호재', 
    '2000-06-02': '호재', '2009-07-15': '호재', '2002-05-15': '호재', '2015-06-04': '호재', '2007-10-26': '호재', 
    '2001-04-06': '호재', '2020-11-16': '호재', '2003-04-16': '호재', '2020-04-17': '호재', '2014-12-19': '호재', 
    '2015-03-02': '호재', '2009-10-09': '호재', '2004-05-12': '호재', '2000-09-19': '호재', '2024-05-07': '호재', 
    '2008-11-26': '호재', '2010-12-02': '호재', '2018-11-02': '호재', '2016-08-18': '호재', '2013-08-13': '호재', 
    '2003-06-17': '호재', '2011-04-20': '호재', '2000-05-02': '호재', '2007-11-22': '호재', '2001-12-27': '호재', 
    '2011-08-23': '호재', '2009-01-06': '호재', '2009-02-06': '호재', '2008-07-24': '호재', '2008-12-19': '호재', 
    '2004-06-24': '호재', '2002-07-23': '호재', '2000-05-15': '호재', '2001-01-19': '호재', '2001-11-19': '호재', 
    '2001-09-04': '호재', '2003-06-19': '호재', '2000-12-05': '호재', '2012-09-07': '호재', '2014-10-30': '호재', 
    '2022-09-13': '호재', '2007-09-27': '호재', '2008-11-21': '호재', '2009-05-28': '호재', '2001-08-01': '호재', 
    '2005-01-17': '호재', '2001-12-12': '호재', '2016-10-06': '호재', '2011-12-21': '호재', '2000-11-14': '호재', 
    '2001-06-08': '호재', '2008-02-04': '호재', '2002-09-17': '호재', '2012-08-06': '호재', '2008-09-08': '호재', 
    '2020-06-16': '호재', '2011-03-14': '호재', '2012-07-13': '호재', '2008-04-25': '호재', '2000-10-24': '호재', 
    '2002-04-17': '호재', '2011-05-02': '호재', '2021-12-01': '호재', '2022-07-15': '호재', '2002-04-15': '호재', 
    '2023-04-07': '호재', '2023-01-04': '호재', '2020-11-23': '호재', '2010-03-17': '호재', '2003-04-18': '호재', 
    '2000-07-26': '호재', '2001-10-19': '호재', '2000-10-16': '호재', '2001-05-29': '호재', '2001-01-18': '호재', 
    '2012-03-09': '호재', '2016-09-13': '호재', '2007-11-29': '호재', '2004-09-10': '호재', '2009-01-02': '호재', 
    '2006-08-17': '호재', '2007-11-15': '호재', '2008-01-31': '호재', '2002-01-25': '호재', '2024-01-19': '호재', 
    '2009-03-11': '호재', '2002-10-15': '호재', '2002-03-27': '호재', '2011-02-14': '호재', '2002-04-23': '호재', 
    '2001-07-12': '호재', '2022-11-11': '호재', '2009-08-14': '호재', '2009-04-09': '호재', '2017-09-18': '호재', 
    '2016-11-30': '호재', '2024-04-24': '호재', '2001-02-08': '호재', '2008-07-17': '호재', '2018-09-14': '호재', 
    '2019-03-21': '호재', '2012-01-19': '호재', '2010-12-03': '호재', '2006-04-21': '호재', '2008-03-19': '호재', 
    '2018-03-06': '호재', '2021-02-25': '호재', '2007-11-07': '호재', '2022-04-29': '호재', '2002-08-07': '호재'}

In [ ]:

# Define your query, keywords, and dates_list
query_list = ["삼성전자", '코스피']
keywords = ["HBM", "파운드리", "QLED", "OLED", "엑시노스", "갤럭시", 
            "이재용", "삼성페이", "메모리", "가우스", "비스포크", '반도체', 
            '평택', '텍사스', '플립', '폴더', '낸드', '패키징', 'bspdn', 'BSPDN', 'GAA']

# 모든 날짜에 대해 뉴스 검색 및 필터링 수행
all_filtered_news = []
for date in tqdm(date_samsung_outcome_mapping.keys(), desc="Processing Samsung News"):
    for query in query_list:
        filtered_news = getFilteredNews(query, date, keywords, display=100)
        for news in filtered_news:
            news['Outcome'] = date_samsung_outcome_mapping.get(date, '')  # 날짜에 맞는 결과를 매핑
            all_filtered_news.append(news)

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(all_filtered_news)


# 엑셀 파일로 저장
output_file_name = "filtered_samsung_news_with_outcome.xlsx"
df.to_excel(output_file_name, index=False, engine='openpyxl')

df.head()  # 첫 몇 줄을 표시하여 결과 확인

Processing Samsung News:   0%|                                                                 | 0/333 [00:00<?, ?it/s]C:\Users\4호실-2\AppData\Local\Temp\ipykernel_1860\1148065935.py:64: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  title = BeautifulSoup(item['title'], 'html.parser').get_text()
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Processing Samsung News:   0%|▏                                                      | 1/333 [00:45<4:09:42, 45.13s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Processing Samsung News:   1%|▎                                                      | 2/333 [01:30<4:09:01, 45.14s/it]Some characters co

In [ ]:
## 애플
apple_date_outcome_mapping = {
    "2000-09-29": "악재", "1983-09-23": "악재", "1987-10-19": "악재", "1993-07-16": "악재",
    "1987-10-26": "악재", "2008-09-29": "악재", "1997-01-06": "악재", "2001-07-18": "악재",
    "2000-12-06": "악재", "2002-06-19": "악재", "1983-11-08": "악재", "1991-05-01": "악재",
    "1991-04-15": "악재", "1981-09-25": "악재", "2020-03-16": "악재", "2002-07-17": "악재",
    "1999-09-21": "악재", "2013-01-24": "악재", "1983-09-07": "악재", "1999-01-14": "악재",
    "2008-01-23": "악재", "1993-06-09": "악재", "1997-12-02": "악재", "1995-09-15": "악재",
    "1997-08-12": "악재", "1985-03-07": "악재", "1988-01-08": "악재", "2019-01-03": "악재",
    "1999-09-23": "악재", "1990-10-16": "악재", "1989-01-27": "악재", "2020-03-12": "악재",
    "1997-10-16": "악재", "1983-07-26": "악재", "1987-10-22": "악재", "2005-04-14": "악재",
    "2008-10-07": "악재", "2000-07-24": "악재", "1998-11-30": "악재", "1982-11-22": "악재",
    "1990-10-03": "악재", "1998-08-28": "악재", "1998-08-31": "악재", "1991-11-15": "악재",
    "2000-01-06": "악재", "2008-09-17": "악재", "2000-04-12": "악재", "1995-12-18": "악재",
    "2000-01-04": "악재", "1983-10-07": "악재", "1997-08-11": "악재", "1983-09-08": "악재",
    "1989-12-12": "악재", "2008-10-02": "악재", "1985-05-24": "악재", "1998-10-05": "악재",
    "1995-03-14": "악재", "2003-04-11": "악재", "1997-08-08": "악재", "2001-03-12": "악재",
    "2020-09-03": "악재", "2014-01-28": "악재", "2000-10-03": "악재", "2000-07-19": "악재",
    "2000-09-22": "악재", "1982-04-13": "악재", "1985-06-03": "악재", "2020-03-09": "악재",
    "1982-01-07": "악재", "2001-09-20": "악재", "2001-05-30": "악재", "1995-12-15": "악재",
    "2001-02-09": "악재", "1981-03-09": "악재", "1985-05-02": "악재", "1981-07-10": "악재",
    "1996-07-03": "악재", "1983-10-18": "악재", "1989-01-17": "악재", "1983-10-17": "악재",
    "1992-07-17": "악재", "1981-07-09": "악재", "1982-03-05": "악재", "2008-01-04": "악재",
    "2000-01-28": "악재", "1998-01-07": "악재", "1995-03-28": "악재", "1981-06-30": "악재",
    "1993-06-07": "악재", "2000-03-30": "악재", "1997-07-01": "악재", "2000-10-24": "악재",
    "1983-07-27": "악재", "1980-12-16": "악재", "1993-01-15": "악재", "1996-06-25": "악재",
    "2002-02-27": "악재", "1987-10-16": "악재", "1981-11-05": "악재", "1982-12-10": "악재",
    "1997-10-20": "악재", "2000-09-21": "악재", "1998-08-27": "악재", "2000-07-28": "악재",
    "2002-11-07": "악재", "1995-01-20": "악재", "2008-10-21": "악재", "1981-11-13": "악재",
    "2002-02-21": "악재", "1995-07-21": "악재", "1985-01-17": "악재", "1995-10-19": "악재",
    "2007-11-12": "악재", "1988-01-20": "악재", "1999-11-09": "악재", "2008-09-22": "악재",
    "1983-06-28": "악재", "1990-04-19": "악재", "1981-08-21": "악재", "2008-11-05": "악재",
    "1983-11-03": "악재", "1998-03-09": "악재", "1985-05-28": "악재", "2001-07-10": "악재",
    "1997-07-21": "악재", "2007-07-31": "악재", "1993-07-19": "악재", "1986-09-11": "악재",
    "1981-07-20": "악재", "2020-09-08": "악재", "2008-11-20": "악재", "2000-05-19": "악재",
    "1997-10-24": "악재", "2018-11-02": "악재", "2000-10-05": "악재", "1999-12-08": "악재",
    "2016-01-27": "악재", "2008-12-17": "악재", "2020-02-27": "악재", "1999-10-19": "악재",
    "1996-01-19": "악재", "1987-11-03": "악재", "1990-07-19": "악재", "2012-12-05": "악재",
    "2008-11-14": "악재", "1998-12-03": "악재", "1992-06-16": "악재", "2000-09-06": "악재",
    "2000-10-17": "악재", "1997-10-17": "악재", "1998-10-01": "악재", "2005-01-11": "악재",
    "2000-03-02": "악재", "2020-03-20": "악재", "2006-02-06": "악재", "1982-02-08": "악재",
    "2003-10-16": "악재", "1989-06-16": "악재", "1982-08-03": "악재", "1982-01-11": "악재",
    "2016-04-27": "악재", "2001-04-03": "악재", "1996-01-30": "악재", "1981-08-24": "악재",
    "1983-07-28": "악재", "1996-01-25": "악재", "2007-01-18": "악재", "1999-05-04": "악재",
    "2000-05-15": "악재", "1982-03-12": "악재", "1987-12-03": "악재", "1989-10-13": "악재",
    "2007-07-24": "악재", "2001-05-03": "악재", "2015-08-21": "악재", "1986-09-02": "악재",
    "1985-02-27": "악재", "1996-01-18": "악재", "1984-11-09": "악재", "2000-11-30": "악재",
    "2001-06-11": "악재", "2000-01-12": "악재", "1997-08-06": "호재", "1998-01-02": "호재",
    "1996-07-18": "호재", "1998-01-06": "호재", "1993-10-15": "호재", "1987-10-29": "호재",
    "1987-10-21": "호재", "1997-07-11": "호재", "1999-10-14": "호재", "1998-10-09": "호재",
    "2008-10-13": "호재", "2000-03-01": "호재", "1984-08-03": "호재", "1983-08-31": "호재",
    "2004-10-14": "호재", "2001-04-19": "호재", "2008-11-24": "호재", "1994-10-05": "호재",
    "2020-03-13": "호재", "2006-07-20": "호재", "1999-04-27": "호재", "2001-04-18": "호재",
    "1994-01-21": "호재", "2003-05-05": "호재", "2004-07-15": "호재", "2004-11-22": "호재",
    "1997-03-27": "호재", "2001-01-18": "호재", "1983-01-20": "호재", "1996-10-01": "호재",
    "2000-01-13": "호재", "1983-10-19": "호재", "1997-08-07": "호재", "2001-01-29": "호재",
    "1987-10-28": "호재", "2000-04-17": "호재", "1994-07-22": "호재", "2007-11-13": "호재",
    "2020-07-31": "호재", "1982-11-30": "호재", "2000-10-13": "호재", "1990-10-19": "호재",
    "2001-01-03": "호재", "2020-03-24": "호재", "2004-04-15": "호재", "2000-06-21": "호재",
    "2006-04-05": "호재", "2000-09-18": "호재", "2000-03-21": "호재", "1999-10-20": "호재",
    "1997-08-01": "호재", "1997-02-18": "호재", "1987-12-14": "호재", "1990-03-16": "호재",
    "1998-09-01": "호재", "2000-09-28": "호재", "2020-03-02": "호재", "1999-07-08": "호재",
    "1980-12-26": "호재", "1999-11-08": "호재", "2005-10-13": "호재", "1994-05-04": "호재",
    "2008-10-10": "호재", "1983-10-12": "호재", "1982-03-19": "호재", "1993-10-21": "호재",
    "1986-05-12": "호재", "1987-01-19": "호재", "1994-02-07": "호재", "1983-09-19": "호재",
    "2022-11-10": "호재", "1998-09-08": "호재", "1998-07-16": "호재", "1983-10-13": "호재",
    "2000-01-03": "호재", "2012-04-25": "호재", "1983-04-20": "호재", "2000-11-02": "호재",
    "1984-01-04": "호재", "2003-05-06": "호재", "1981-04-02": "호재", "2020-04-06": "호재",
    "2001-01-11": "호재", "1998-03-11": "호재", "1991-03-26": "호재", "1996-01-05": "호재",
    "2008-10-28": "호재", "2002-05-08": "호재", "1983-10-31": "호재", "1998-12-22": "호재",
    "1981-12-17": "호재", "2007-01-09": "호재", "1981-12-18": "호재", "1997-10-28": "호재",
    "2014-04-24": "호재", "1981-10-02": "호재", "2002-01-17": "호재", "1991-03-05": "호재",
    "2001-07-12": "호재", "2002-03-01": "호재", "1995-01-06": "호재", "1987-10-27": "호재",
    "1982-10-07": "호재", "2000-03-31": "호재", "2008-09-30": "호재", "1982-03-18": "호재",
    "1985-12-18": "호재", "1987-01-14": "호재", "1985-06-04": "호재", "1999-08-11": "호재",
    "1999-01-07": "호재", "2001-06-25": "호재", "1999-05-03": "호재", "1999-04-23": "호재",
    "2001-11-26": "호재", "1987-09-22": "호재", "1983-11-09": "호재", "1984-04-26": "호재",
    "1992-10-16": "호재", "2010-05-10": "호재", "2022-10-28": "호재", "1990-10-18": "호재",
    "1982-03-22": "호재", "1987-04-02": "호재", "2001-03-22": "호재", "1982-10-08": "호재",
    "1982-11-03": "호재", "2001-06-20": "호재", "1996-05-06": "호재", "1987-12-07": "호재",
    "1983-01-14": "호재", "2001-04-10": "호재", "1982-10-06": "호재", "2005-01-07": "호재",
    "2024-06-11": "호재", "2012-11-19": "호재", "2020-03-10": "호재", "1999-09-30": "호재",
    "1998-03-24": "호재", "1987-01-22": "호재", "1994-01-05": "호재", "1984-05-21": "호재",
    "1987-02-11": "호재", "1997-08-13": "호재", "2018-12-26": "호재", "1991-11-08": "호재",
    "2001-04-05": "호재", "2008-11-13": "호재", "1995-01-11": "호재", "1982-11-02": "호재",
    "1985-04-10": "호재", "2022-01-28": "호재", "1997-03-20": "호재", "1999-04-30": "호재",
    "1982-08-25": "호재", "1985-06-24": "호재", "2002-05-14": "호재", "1987-08-06": "호재",
    "1999-12-02": "호재", "1983-01-27": "호재", "1985-05-16": "호재", "2000-03-22": "호재",
    "1998-12-01": "호재", "1998-01-13": "호재", "1984-08-06": "호재", "1987-02-17": "호재",
    "2019-01-30": "호재", "1981-04-21": "호재", "2002-07-05": "호재", "2007-10-23": "호재",
    "1998-07-08": "호재", "1995-06-20": "호재", "2009-05-26": "호재", "1991-03-07": "호재",
    "1993-10-27": "호재", "2004-06-16": "호재", "2009-01-22": "호재", "1982-01-15": "호재",
    "1996-04-09": "호재", "2000-12-22": "호재", "2009-03-10": "호재", "2005-01-13": "호재",
    "2001-07-11": "호재", "1999-06-29": "호재", "1986-03-20": "호재", "1998-10-12": "호재",
    "1987-07-13": "호재", "1985-09-18": "호재", "1983-05-05": "호재", "1988-01-04": "호재",
    "1996-09-16": "호재", "1982-07-20": "호재", "1982-08-17": "호재", "2000-01-20": "호재",
    "1983-02-10": "호재", "2016-07-27": "호재", "2000-02-14": "호재", "2001-01-23": "호재",
    "2000-04-25": "호재", "1997-07-17": "호재", "1982-07-13": "호재", "1982-11-11": "호재",
    "1991-01-16": "호재", "2001-12-04": "호재", "1995-05-12": "호재", "2008-03-11": "호재",
    "2002-01-02": "호재", "1999-03-29": "호재", "1988-04-06": "호재", "2007-07-26": "호재",
    "2001-03-23": "호재", "2020-10-12": "호재", "1990-08-27": "호재", "2009-01-02": "호재",
    "2006-01-10": "호재", "1981-02-25": "호재", "1994-10-12": "호재", 
    "1985-10-30": "호재", "1983-10-24": "호재", "2004-03-05": "호재",
    "2005-07-14": "호재", "1998-02-23": "호재", "1993-11-16": "호재", "1988-01-11": "호재",
    "2012-01-25": "호재", "1991-08-01": "호재", "1981-09-01": "호재", "2008-10-30": "호재",
    "1991-04-02": "호재", "1983-05-04": "호재", "1981-03-18": "호재", "1986-01-29": "호재",
    "1999-05-26": "호재", "1987-07-14": "호재", "1991-04-11": "호재", "2006-03-29": "호재",
    "2001-04-27": "호재", "1980-12-19": "호재", "2000-06-01": "호재", "2017-02-01": "호재",
    "2000-09-07": "호재", "2008-12-08": "호재", "2001-12-05": "호재", "1984-11-06": "호재",
    "1982-08-10": "호재", "2000-08-17": "호재", "1981-01-19": "호재", "1987-01-27": "호재",
    "1995-01-10": "호재", "2004-11-29": "호재", "1984-12-18": "호재", "2001-10-04": "호재",
    "1990-10-17": "호재", "2006-04-06": "호재"
}

In [ ]:
# 네이버 뉴스 검색 쿼리와 키워드 정의
query = "애플"
keywords = [
    "M1", "M2", "M3", "아이폰", "A17", "Apple Intelligence", 
    "비젼 프로", "비전 프로", "ios", "팀 쿡", 
    "앱스토어", "애플티비", "애플뮤직"
]

# 모든 날짜에 대해 뉴스 검색 및 필터링 수행
all_filtered_news = []
for date in tqdm(apple_date_outcome_mapping.keys(), desc="Processing Apple News"):
    filtered_news = getFilteredNews(query, date.replace("-", ""), keywords, display=100)
    for news in filtered_news:
        news['Outcome'] = apple_date_outcome_mapping.get(date, '')  # 날짜에 맞는 결과를 매핑
        all_filtered_news.append(news)

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(all_filtered_news)

# 엑셀 파일로 저장
output_file_name = "filtered_apple_news_with_outcome.xlsx"
df.to_excel(output_file_name, index=False, engine='openpyxl')

df.head()  # 첫 몇 줄을 표시하여 결과 확인

In [ ]:
Bitcoin_date_outcome_mapping = {
    "2020-03-12": "악재", "2015-01-14": "악재", "2017-09-14": "악재", "2015-08-18": "악재",
    "2018-01-16": "악재", "2022-06-13": "악재", "2018-02-05": "악재", "2015-01-13": "악재",
    "2016-01-15": "악재", "2022-11-09": "악재", "2017-01-11": "악재", "2019-06-27": "악재",
    "2021-05-19": "악재", "2018-11-19": "악재", "2021-05-12": "악재", "2021-01-21": "악재",
    "2019-07-16": "악재", "2017-12-22": "악재", "2017-01-05": "악재", "2017-12-30": "악재",
    "2017-03-18": "악재", "2015-11-10": "악재", "2019-09-24": "악재", "2021-06-21": "악재",
    "2015-01-28": "악재", "2021-09-07": "악재", "2022-05-09": "악재", "2017-01-06": "악재",
    "2015-01-03": "악재", "2018-11-24": "악재", "2016-06-22": "악재", "2018-01-30": "악재",
    "2017-07-15": "악재", "2018-01-11": "악재", "2022-01-21": "악재", "2018-02-01": "악재",
    "2020-09-03": "악재", "2015-03-18": "악재", "2017-06-12": "악재", "2018-01-21": "악재",
    "2018-03-14": "악재", "2022-08-19": "악재", "2022-11-08": "악재", "2019-07-14": "악재",
    "2021-02-23": "악재", "2018-03-29": "악재", "2018-06-10": "악재", "2018-02-04": "악재",
    "2018-11-14": "악재", "2017-03-24": "악재", "2022-06-16": "악재", "2016-08-02": "악재",
    "2018-06-22": "악재", "2016-06-21": "악재", "2019-06-30": "악재", "2021-09-20": "악재",
    "2022-09-13": "악재", "2020-03-08": "악재", "2019-01-10": "악재", "2018-04-25": "악재",
    "2015-02-15": "악재", "2020-05-10": "악재", "2021-06-25": "악재", "2018-11-20": "악재",
    "2021-05-21": "악재", "2014-10-04": "악재", "2020-11-26": "악재", "2015-12-26": "악재",
    "2017-12-09": "악재", "2024-03-19": "악재", "2021-12-04": "악재", "2018-04-04": "악재",
    "2017-09-08": "악재", "2019-02-24": "악재", "2018-01-08": "악재", "2015-03-24": "악재",
    "2022-05-05": "악재", "2022-03-04": "악재", "2022-02-17": "악재", "2017-12-28": "악재",
    "2017-06-14": "악재", "2019-08-14": "악재", "2015-08-24": "악재", "2018-09-05": "악재",
    "2015-11-11": "악재", "2024-01-12": "악재", "2017-09-04": "악재", "2018-03-07": "악재",
    "2017-12-08": "악재", "2017-05-27": "악재", "2017-03-17": "악재", "2021-05-23": "악재",
    "2021-04-18": "악재", "2017-11-10": "악재", "2021-01-11": "악재", "2014-09-18": "악재",
    "2021-05-28": "악재", "2022-06-18": "악재", "2024-08-05": "악재", "2023-08-17": "악재",
    "2017-09-21": "악재", "2016-11-03": "악재", "2020-03-16": "악재", "2017-12-19": "악재",
    "2014-09-19": "악재", "2019-10-23": "악재", "2019-05-17": "악재", "2018-03-06": "악재",
    "2017-10-24": "악재", "2016-01-22": "악재", "2021-05-04": "악재", "2022-05-11": "악재",
    "2017-05-12": "악재", "2021-12-13": "악재", "2018-05-11": "악재", "2015-08-08": "악재",
    "2024-03-05": "악재", "2018-08-08": "악재", "2019-07-11": "악재", "2021-01-27": "악재",
    "2020-03-14": "악재", "2017-12-20": "악재", "2017-07-25": "악재", "2021-11-26": "악재",
    "2017-02-09": "악재", "2014-10-23": "악재", "2018-12-25": "악재", "2021-06-07": "악재",
    "2018-02-22": "악재", "2017-11-12": "악재", "2017-09-02": "악재", "2016-07-03": "악재",
    "2017-03-22": "악재", "2022-04-11": "악재", "2021-05-17": "악재", "2020-06-02": "악재",
    "2022-06-01": "악재", "2021-05-15": "악재", "2018-02-21": "악재", "2023-03-09": "악재",
    "2019-07-04": "악재", "2022-08-26": "악재", "2018-12-06": "악재", "2014-10-29": "악재",
    "2015-11-05": "악재", "2018-11-30": "악재", "2018-07-10": "악재", "2019-06-03": "악재",
    "2019-06-04": "악재", "2024-04-02": "악재", "2022-03-10": "악재", "2015-07-13": "악재",
    "2017-03-10": "악재", "2024-08-02": "악재", "2021-01-15": "악재", "2021-12-28": "악재",
    "2018-05-23": "악재", "2017-03-08": "악재", "2017-09-13": "악재", "2015-01-04": "악재",
    "2020-08-02": "악재",
    "2017-12-07": "호재", "2017-07-20": "호재", "2017-12-06": "호재", "2021-02-08": "호재",
    "2020-03-19": "호재", "2015-01-15": "호재", "2019-04-02": "호재", "2019-10-25": "호재",
    "2017-07-17": "호재", "2017-09-15": "호재", "2014-11-12": "호재", "2017-12-26": "호재",
    "2022-02-28": "호재", "2017-09-18": "호재", "2018-04-12": "호재", "2019-05-11": "호재",
    "2017-10-12": "호재", "2019-05-19": "호재", "2020-04-29": "호재", "2017-08-05": "호재",
    "2024-08-08": "호재", "2019-05-13": "호재", "2016-05-28": "호재", "2020-03-13": "호재",
    "2018-01-05": "호재", "2022-02-04": "호재", "2015-11-03": "호재", "2021-06-09": "호재",
    "2018-02-06": "호재", "2018-11-28": "호재", "2021-05-24": "호재", "2018-01-20": "호재",
    "2015-11-02": "호재", "2020-07-27": "호재", "2019-06-28": "호재", "2016-06-12": "호재",
    "2022-11-10": "호재", "2019-07-03": "호재", "2017-01-04": "호재", "2022-09-09": "호재",
    "2016-01-20": "호재", "2018-02-14": "호재", "2019-06-26": "호재", "2018-12-20": "호재",
    "2016-06-16": "호재", "2023-10-23": "호재", "2017-11-15": "호재", "2017-11-13": "호재",
    "2021-04-26": "호재", "2020-11-05": "호재", "2021-05-20": "호재", "2017-12-16": "호재",
    "2020-03-23": "호재", "2019-07-18": "호재", "2021-01-13": "호재", "2021-06-13": "호재",
    "2021-01-28": "호재", "2021-03-01": "호재", "2021-10-01": "호재", "2018-02-12": "호재",
    "2020-12-16": "호재", "2018-01-02": "호재", "2024-03-20": "호재", "2017-12-11": "호재",
    "2019-12-18": "호재", "2024-02-28": "호재", "2023-03-17": "호재", "2023-02-15": "호재",
    "2021-01-02": "호재", "2015-02-14": "호재", "2017-12-31": "호재", "2018-07-24": "호재",
    "2017-01-17": "호재", "2023-03-13": "호재", "2018-12-17": "호재", "2015-12-11": "호재",
    "2015-11-12": "호재", "2018-07-17": "호재", "2018-02-17": "호재", "2018-04-24": "호재",
    "2020-03-30": "호재", "2018-02-08": "호재", "2020-01-14": "호재", "2022-03-09": "호재",
    "2014-09-23": "호재", "2021-01-06": "호재", "2022-07-18": "호재", "2017-03-27": "호재",
    "2024-03-04": "호재", "2021-02-19": "호재", "2022-06-19": "호재", "2018-03-11": "호재",
    "2020-11-30": "호재", "2017-08-11": "호재", "2022-07-27": "호재", "2017-05-08": "호재",
    "2017-09-27": "호재", "2019-02-08": "호재", "2021-04-30": "호재", "2024-05-20": "호재",
    "2015-01-26": "호재", "2022-05-30": "호재", "2021-07-21": "호재", "2019-05-26": "호재",
    "2021-05-05": "호재", "2024-05-15": "호재", "2021-06-27": "호재", "2019-08-05": "호재",
    "2020-10-21": "호재", "2017-11-16": "호재", "2021-08-13": "호재", "2023-01-20": "호재",
    "2021-10-06": "호재", "2020-06-01": "호재", "2015-11-26": "호재", "2021-10-15": "호재",
    "2023-03-12": "호재", "2015-08-19": "호재", "2015-01-21": "호재", "2020-05-07": "호재",
    "2018-12-28": "호재", "2015-02-27": "호재", "2019-07-08": "호재", "2018-02-26": "호재",
    "2017-12-01": "호재", "2020-12-26": "호재", "2020-04-16": "호재", "2015-12-05": "호재",
    "2019-06-25": "호재", "2021-09-22": "호재", "2020-04-06": "호재", "2018-02-15": "호재",
    "2021-01-22": "호재", "2020-12-17": "호재", "2017-06-05": "호재", "2017-10-29": "호재",
    "2021-01-07": "호재", "2017-12-15": "호재", "2024-01-08": "호재", "2021-03-13": "호재",
    "2017-05-23": "호재", "2019-10-26": "호재", "2016-01-07": "호재", "2024-07-15": "호재",
    "2021-11-08": "호재", "2021-02-11": "호재", "2018-04-20": "호재", "2016-12-23": "호재",
    "2021-03-26": "호재", "2016-06-24": "호재", "2017-09-25": "호재", "2023-11-15": "호재",
    "2019-02-18": "호재", "2017-06-06": "호재", "2017-11-25": "호재", "2019-06-21": "호재",
    "2017-05-22": "호재", "2017-03-19": "호재", "2015-01-24": "호재", "2017-08-12": "호재",
    "2018-02-19": "호재", "2024-05-03": "호재", "2016-01-16": "호재", "2021-01-05": "호재",
    "2017-03-21": "호재", "2017-12-23": "호재", "2019-07-15": "호재", "2023-08-29": "호재",
    "2017-08-14": "호재", "2015-02-13": "호재", "2017-11-26": "호재", "2015-07-12": "호재",
    "2019-09-02": "호재", "2017-05-17": "호재", "2021-04-13": "호재", "2019-01-06": "호재",
    "2018-07-16": "호재", "2023-06-21": "호재"
    
}

In [ ]:
# 네이버 뉴스 검색 쿼리와 키워드 정의
query = "비트코인"
keywords = [
    "엘살바도르", "비트", "블록체인", "빗썸",
    "채굴", "그래픽 카드", "트럼프", "암호화폐", "탈중앙화",
    "알트", "이더리움", "래플", "도지", "루나"
]

# 모든 날짜에 대해 뉴스 검색 및 필터링 수행
all_filtered_news = []
for date in tqdm(Bitcoin_date_outcome_mapping.keys(), desc="Processing Bitcoin News"):
    filtered_news = getFilteredNews(query, date.replace("-", ""), keywords, display=100)
    for news in filtered_news:
        news['Outcome'] = Bitcoin_date_outcome_mapping.get(date, '')  # 날짜에 맞는 결과를 매핑
        all_filtered_news.append(news)

# 결과를 데이터프레임으로 변환
df = pd.DataFrame(all_filtered_news)

# 엑셀 파일로 저장
output_file_name = "filtered_Bitcoin_news_with_outcome.xlsx"
df.to_excel(output_file_name, index=False, engine='openpyxl')

df.head()  # 첫 몇 줄을 표시하여 결과 확인